In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import pandas as pd
data = pd.read_csv('prev_exam.csv')
data.head()

,Unnamed: 0,1,2,3,4,5,label
0,0,구슬 탈출 2,스타트링크에서 판매하는 어린이용 장난감 중에서 가장 인기가 많은 제품은 구슬 탈출이...,14611,85456,27.741%,0
1,1,2048 (Easy),2048 게임은 4×4 크기의 보드에서 혼자 즐기는 재미있는 게임이다. 이 링크를 ...,14443,84385,26.328%,0
2,2,뱀,"'Dummy' 라는 도스게임이 있다. 이 게임에는 뱀이 나와서 기어다니는데, 사과를...",19643,69605,40.460%,0
3,3,시험 감독,"총 N개의 시험장이 있고, 각각의 시험장마다 응시자들이 있다. i번 시험장에 있는 ...",20013,89085,29.030%,0
4,4,주사위 굴리기,"크기가 N×M인 지도가 존재한다. 지도의 오른쪽은 동쪽, 위쪽은 북쪽이다. 이 지도...",15652,47791,44.702%,0


In [3]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 319
test shape is: 80


In [4]:
texts = list(data['2'])
labels = list(data['label'])
rate = list(data['5'])

In [5]:
import torch
encoding = tokenizer(texts, rate, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(labels)


In [6]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)


In [7]:
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',  # Pad to the maximum length in the batch
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label),
        }

# Create DataLoader
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [8]:
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in dataloader:
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)
            all_labels.extend(batch["labels"].numpy())
            all_predictions.extend(predictions.numpy())

    return all_labels, all_predictions

In [9]:
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score

# # Define the classifier on top of KoBERT
# classifier = nn.Linear(model.config.hidden_size, 2)  # Assuming binary classification

# # Combine KoBERT model with classifier
# model.classifier = classifier

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# Train the model
for epoch in range(3):  # Adjust the number of epochs as needed
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch["labels"])
        loss.backward()
        optimizer.step()

    all_labels, all_predictions = evaluate(model, test_loader)

    # Calculate accuracy and ROC-AUC
    accuracy = accuracy_score(all_labels, all_predictions)
    roc_auc = roc_auc_score(all_labels, all_predictions)

    print(f"Epoch {epoch + 1} - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")


<ipython-input-7-0054740794e9>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Epoch 1 - Accuracy: 0.7125, ROC-AUC: 0.5000


<ipython-input-7-0054740794e9>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Epoch 2 - Accuracy: 0.3250, ROC-AUC: 0.5263


<ipython-input-7-0054740794e9>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Epoch 3 - Accuracy: 0.7125, ROC-AUC: 0.5000


In [10]:
final_all_labels, final_all_predictions = evaluate(model, test_loader)
final_accuracy = accuracy_score(final_all_labels, final_all_predictions)
final_roc_auc = roc_auc_score(final_all_labels, final_all_predictions)

print(f"Final Accuracy: {final_accuracy:.4f}, Final ROC-AUC: {final_roc_auc:.4f}")

<ipython-input-7-0054740794e9>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(label),


Final Accuracy: 0.7125, Final ROC-AUC: 0.5000


In [11]:
torch.save(model, "model.pt")